<a href="https://colab.research.google.com/github/raj-arun/epm_rest/blob/main/Add_Member_in_EPM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Add the packages needed for the program**
More details on requests library can be found [here](https://docs.python-requests.org/en/master/)

In [1]:
import requests
import json
from requests.auth import HTTPBasicAuth



---


Read the json file.

---



In [2]:
# read the data file containing login information
logindata = {}
with open('instanceDetails.json') as f:
   logindata = json.load(f)



---


Get the user name, url and password from the file


---



In [3]:
# variables to pass login information while invoking the REST API
uName = logindata["user"]
passwd = logindata["pwd"]
epmUrl = logindata["instance"]

# variables to hold version information and app name
apiVersion = ""
appName = ""

In [4]:
# construct api end point and assign request headers
requestURL = epmUrl + "/HyperionPlanning/rest/"

reqHeaders = {}
reqHeaders['Content-Type'] = 'application/json'

---
1.  [GET REST API](https://docs.oracle.com/en/cloud/saas/enterprise-performance-management-common/prest/get_rest_api_versions_for_planning.html) Versions API
2.  GET Method
3.  Retrieve the version information
---

In [ ]:
# invoke the rest api, check response
reqResponse = requests.get(requestURL, auth=HTTPBasicAuth(uName, passwd), headers=reqHeaders)

if reqResponse.status_code == 200:
    versionData = json.loads(reqResponse.text)
    print(json.dumps(versionData, indent=4, sort_keys=True))
else:
    print(reqResponse.status_code)

In [6]:
# loop through the response and get the version information
for i in versionData["items"]:
    if(i["isLatest"]):
        apiVersion = i["version"]
        
print("API Version is {}".format(apiVersion))

API Version is v3


---
1.  [GET Application Details](https://docs.oracle.com/en/cloud/saas/enterprise-performance-management-common/prest/get_applications.html) API
2.  GET Method
3.  Retrieve the application name, type etc
---

In [ ]:
# construct api end point, invoke the rest api, check response
appURL = epmUrl + "/HyperionPlanning/rest/"+apiVersion+"/applications"
reqResponse = requests.get(appURL, auth=HTTPBasicAuth( uName, passwd), headers=reqHeaders)

if reqResponse.status_code == 200:
    appData = json.loads(reqResponse.text)
    print(json.dumps(appData, indent=4, sort_keys=True))
else:
    print("Error invoking REST API. Error Code: {1}".format(reqResponse.status_code))

In [ ]:
# loop through the response and get the version information
for i in appData["items"]:
    appName = i["name"]
    
print("App Name is {}".format(appName))

---
1.  [Add Member](https://docs.oracle.com/en/cloud/saas/enterprise-performance-management-common/prest/add_member.html) API
2.  POST Method
3.  Limited set of attributes are available
4.  The parent member must be enabled for dynamic children and a cube refresh must have happened after the parent was enabled.
---

In [ ]:
dimURL = appURL + "/"+ appName + "/dimensions/Account/members"
print(dimURL)
payload = json.dumps({"memberName":"100099","parentName":"100011"})
reqResponse = requests.post(dimURL, auth=HTTPBasicAuth( uName, passwd), headers=reqHeaders,data=payload)

if reqResponse.status_code == 200 or reqResponse.status_code == 201:
    appData = json.loads(reqResponse.text)
    print(json.dumps(appData, indent=4, sort_keys=True))
else:
    #print("Error invoking REST API. Error Code: {1}".format(reqResponse.status_code))
    print(reqResponse.text)

---
1.  [Execute Job](https://docs.oracle.com/en/cloud/saas/enterprise-performance-management-common/prest/execute_a_job.html) API
2.  POST Method
3.  Import Data / Metadata, Rules, Cube Refresh and many more..
---

In [ ]:
#submit job
jobURL = appURL + "/"+ appName + '/jobs'
print(jobURL)
payload = json.dumps({"jobType":"Cube Refresh","jobName":"Refresh Database"})
reqResponse = requests.post(jobURL, auth=HTTPBasicAuth( uName, passwd), headers=reqHeaders,data=payload)

if reqResponse.status_code == 200 or reqResponse.status_code == 201:
    appData = json.loads(reqResponse.text)
    print(json.dumps(appData, indent=4, sort_keys=True))
else:
    #print("Error invoking REST API. Error Code: {1}".format(reqResponse.status_code))
    print(reqResponse.text)

---
1.  [Get Job Status](https://docs.oracle.com/en/cloud/saas/enterprise-performance-management-common/prest/retrieve_job_status.html) API
2.  GET Method
3.  Job Identifier is needed
---

In [ ]:
# job details
# loop through the response and get the version information
for i in appData["links"]:
    if (i["rel"] == "self"):
      jobDetailsURL = i["href"]
    
print("Job Details URL {}".format(jobDetailsURL))

reqResponse = requests.get(jobDetailsURL, auth=HTTPBasicAuth( uName, passwd), headers=reqHeaders)

if reqResponse.status_code == 200 or reqResponse.status_code == 201:
    appData = json.loads(reqResponse.text)
    print(json.dumps(appData, indent=4, sort_keys=True))
else:
    #print("Error invoking REST API. Error Code: {1}".format(reqResponse.status_code))
    print(reqResponse.text)

---
- [SendMail](https://docs.oracle.com/en/cloud/saas/enterprise-performance-management-common/prest/lcm_send_email.html) REST API
- Content Type is application/x-www-form-urlencoded
- POST Method
- Attachements can be included
---


In [ ]:
#sendmail
reqHeaders = {}
reqHeaders['Content-Type'] = 'application/x-www-form-urlencoded'
migration_api_version = "v1"

# construct api end point and assign request headers
migrationURL = epmUrl + "/interop/rest/"+migration_api_version+"/services/sendmail"
print(migrationURL)

payload = 'to=s.arun.raj%40gmail.com&subject=Email%20from%20REST%20API&body=some%20body%20test'
reqResponse = requests.post(migrationURL, auth=HTTPBasicAuth( uName, passwd), headers=reqHeaders,data=payload)

if reqResponse.status_code == 200 or reqResponse.status_code == 201:
    appData = json.loads(reqResponse.text)
    print(json.dumps(appData, indent=4, sort_keys=True))
else:
    print(reqResponse.text)

---
- [Delete Files](https://docs.oracle.com/en/cloud/saas/enterprise-performance-management-common/prest/delete_files.html) REST API
- Delete the file from EPM
- DELETE method
---

In [ ]:
#deletefile
reqHeaders = {}
reqHeaders['Content-Type'] = 'application/json'
migration_api_version = "11.1.2.3.600"
fileName = "account_metadata.csv"
# construct api end point and assign request headers
deleteFileURL = epmUrl + "/interop/rest/"+migration_api_version+"/applicationsnapshots/"+fileName
print(deleteFileURL)

reqResponse = requests.delete(deleteFileURL, auth=HTTPBasicAuth( uName, passwd), headers=reqHeaders)

if reqResponse.status_code == 200 or reqResponse.status_code == 201:
    appData = json.loads(reqResponse.text)
    print(json.dumps(appData, indent=4, sort_keys=True))
else:
    print(reqResponse.text)

---
- [List Files](https://colab.research.google.com/drive/1DYTrIxyoJc1snu8RHd-UpWbpbsLuvdN2#scrollTo=9k8wlNytFQcH&line=3&uniqifier=1) REST API
- Loop through the files
- Check for the file and get the file size
- GET Method
---

In [7]:
def findFile(fileName):
  reqHeaders = {}
  reqHeaders['Content-Type'] = 'application/json'
  migration_api_version = "11.1.2.3.600"
  # construct api end point and assign request headers
  listFileURL = epmUrl + "/interop/rest/"+migration_api_version+"/applicationsnapshots"
  #print(listFileURL)

  reqResponse = requests.get(listFileURL, auth=HTTPBasicAuth( uName, passwd), headers=reqHeaders)

  if reqResponse.status_code == 200 or reqResponse.status_code == 201:
      fileList = json.loads(reqResponse.text)
      #print(json.dumps(fileList, indent=4, sort_keys=True))
  else:
      print(reqResponse.text)

  for f in fileList["items"]:
    if f["name"] == fileName:
      print("Found file {}".format(f["name"]))
      print("File size is {} bytes".format(f["size"]))


In [8]:
findFile("Movements.csv")

Found file Movements.csv
File size is 18503 bytes
